# Continuous Optimization: Linear Programming

In [11]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.easy_install()
else:
    sys.path.insert(0, '../')
    import helper
helper.set_plotting_style()

--2024-10-28 03:14:30--  https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6493 (6.3K) [text/plain]
Saving to: ‘helper.py.1’

helper.py.1         100%[===================>]   6.34K  --.-KB/s    in 0s      

2024-10-28 03:14:30 (55.9 MB/s) - ‘helper.py.1’ saved [6493/6493]

idaes was found! No need to install.


In [12]:
import pandas as pd
import pyomo.environ as pyo
import idaes

## Linear Programs: Student Diet Example

Reference: https://docs.mosek.com/modeling-cookbook/linear.html

You want to save money eating while remaining healthy. A healthy diet requires at least P=6 units of protein, C=15 units of carbohydrates, F=5 units of fats and V=7 units of vitamins. Due to compounding factors (blizzard during Lent), our campus only has these options:

In [13]:
# Load data from file, use the first column (0, recall Python starts counting at 0) as the index
food_options = pd.read_csv('https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/data/student_diet.csv',index_col=0)

# Print up the the first 10 rows of data
food_options.head(10)

,P,C,F,V,price
takeaway,3.0,3,2,1,5
vegetables,1.0,2,0,4,1
bread,0.5,4,1,0,2


Let's build a Python dictionary to store the nutrient requirements. (I strongly recommend not touching Python until we write the model on paper. I am including this here to avoid scrolling between the problem description and this cell.)

In [20]:
# Uncomment and fill in with all of the data
nutrient_requirements = {'P':6, 'C':15 }

# Add your solution here

### Propose an Optimization Model

**Sets**

```{dropdown} Click to expand
$$
\mathcal{F} = \{\text{take away, veggies, bread}\}
$$

$$
\mathcal{N} = \{P, C, F, V\}
$$
```

**Parameters**

```{dropdown} Click to expand
$\pi_i$: price for food option $i \in \mathcal{F}$

$r_j$: nutrition requirement for nutrient $j \in \mathcal{N}$

$D_{ij}$: nutrition info for food $i \in \mathcal{F}$ and nutrient $j \in \mathcal{N}$

```

**Variables**

```{dropdown} Click to expand
$x_i$: amount of food $i \in \mathcal{F}$ eaten/purchased
```

**Objective and Constraints**

```{dropdown} Click to expand

Minimize:
$
\sum_{i \in \mathcal{F}} x_i \cdot \pi_i \quad \text{(cost)}
$

Subject to:
$
\sum_{i \in \mathcal{F}} D_{ij} \cdot x_i \geq r_j \quad \forall j \in \mathcal{N} \quad \text{(be healthy)}
$

Bounds:
$
x_i \geq 0 \quad \forall i \in \mathcal{F}
$
```

**Full Model**

Let's put all of the components together into the full model written in compact notation.

$$
\begin{align*}
\min_{x} \quad & \sum_{i \in \mathcal{F}} x_i \cdot \pi_i & \text{(cost)} \\
\text{s.t.} \quad & \sum_{i \in \mathcal{F}} D_{ij} \cdot x_i \geq r_j, \quad \forall j \in \mathcal{N} & \text{(be healthy)} \\
& x_i \geq 0, \quad \forall i \in \mathcal{F} & \text{(non-negative bounds)}
\end{align*}
$$

**Degree of Freedom Analysis**

We will later learn more about how to factor inequality constraints into degree of freedom analysis. For now, please count the number of equality and inequality constraints separately.

```{dropdown} Click to expand

Continuous variables: 3

Inequality constraints: 4

Bounds: 3
```

*Question:* Why is it okay to have more inequality constraints than variables?

*Answer:* Not all inequality constraints may be active at a time.

### Solve in Pyomo

With our optimization model written on paper, we can proceed to solve in Pyomo. Before we start, let's review a few pandas tricks.

In [14]:
# Extract the column names, convert to a list
food_options.columns.to_list()

['P', 'C', 'F', 'V', 'price']

In [15]:
# Same as above, but drop the last entry
nutrients = food_options.columns.to_list()[0:4]
nutrients

['P', 'C', 'F', 'V']

In [16]:
# Extract the index names, convert to a list
foods = food_options.index.to_list()
foods

['takeaway', 'vegetables', 'bread']

In [17]:
# Create a dictionary with keys such as ('takeaway', 'P')
# Do not include 'price'
food_info = food_options[nutrients].stack().to_dict()
food_info

{('takeaway', 'P'): 3.0,
 ('takeaway', 'C'): 3.0,
 ('takeaway', 'F'): 2.0,
 ('takeaway', 'V'): 1.0,
 ('vegetables', 'P'): 1.0,
 ('vegetables', 'C'): 2.0,
 ('vegetables', 'F'): 0.0,
 ('vegetables', 'V'): 4.0,
 ('bread', 'P'): 0.5,
 ('bread', 'C'): 4.0,
 ('bread', 'F'): 1.0,
 ('bread', 'V'): 0.0}

In [18]:
# Create dictionary of only prices
price = food_options['price'].to_dict()
price

{'takeaway': 5, 'vegetables': 1, 'bread': 2}

In [22]:

# Define nutrient requirements (ensure all nutrients are covered)
nutrient_requirements = {
    'P': 6,    # Protein requirement
    'C': 15,   # Carbohydrates requirement
    'F': 8,    # Fats requirement
    'V': 10    # Vitamins requirement
}



Now let's build our Pyomo model!

In [23]:
# Add your solution here
m = pyo.ConcreteModel()

## Define sets
m.FOOD = pyo.Set(initialize = foods)
m.NUTRIENTS = pyo.Set(initialize = nutrients)

## Define parameters
m.needs = pyo.Param(m.NUTRIENTS, initialize=nutrient_requirements)
m.food_info = pyo.Param(m.FOOD, m.NUTRIENTS, initialize=food_info)
m.price = pyo.Param(m.FOOD, initialize=price)

## Define variables
m.food_eaten = pyo.Var(m.FOOD,initialize=1.0,domain=pyo.NonNegativeReals)

## Define constraints
def diet_min_rule(m, n):
    return sum(m.food_info[f,n] * m.food_eaten[f] for f in m.FOOD) >= m.needs[n]

m.diet_min = pyo.Constraint(m.NUTRIENTS, rule=diet_min_rule)

## Define objective
m.cost = pyo.Objective(expr=sum(m.food_eaten[f]*m.price[f] for f in m.FOOD), sense=pyo.minimize)

# Print model
m.pprint()

2 Set Declarations
    FOOD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'takeaway', 'vegetables', 'bread'}
    NUTRIENTS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'P', 'C', 'F', 'V'}

3 Param Declarations
    food_info : Size=12, Index=FOOD*NUTRIENTS, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             ('bread', 'C') :   4.0
             ('bread', 'F') :   1.0
             ('bread', 'P') :   0.5
             ('bread', 'V') :   0.0
          ('takeaway', 'C') :   3.0
          ('takeaway', 'F') :   2.0
          ('takeaway', 'P') :   3.0
          ('takeaway', 'V') :   1.0
        ('vegetables', 'C') :   2.0
        ('vegetables', 'F') :   0.0
        ('vegetables', 'P') :   1.0
        ('vegetables', 'V') :   4.0
    needs : Size=4, Index=NUTRIENTS, Domain=Any, Default=None, Mutable=F

```{dropdown} Click to see the solution to the activity

```python
m = pyo.ConcreteModel()

## Define sets
m.FOOD = pyo.Set(initialize = foods)
m.NUTRIENTS = pyo.Set(initialize = nutrients)

## Define parameters
m.needs = pyo.Param(m.NUTRIENTS, initialize=nutrient_requirements)
m.food_info = pyo.Param(m.FOOD, m.NUTRIENTS, initialize=food_info)
m.price = pyo.Param(m.FOOD, initialize=price)

## Define variables
m.food_eaten = pyo.Var(m.FOOD,initialize=1.0,domain=pyo.NonNegativeReals)

## Define constraints
def diet_min_rule(m, n):
    return sum(m.food_info[f,n] * m.food_eaten[f] for f in m.FOOD) >= m.needs[n]

m.diet_min = pyo.Constraint(m.NUTRIENTS, rule=diet_min_rule)

## Define objective
m.cost = pyo.Objective(expr=sum(m.food_eaten[f]*m.price[f] for f in m.FOOD), sense=pyo.minimize)

# Print model
m.pprint()

<div class="admonition note">
<p class="title"><b>Activity</b></p>
 Check the Pyomo model. Specifically, are the input (parameter) data correct? Do the equations match our model written on paper?
</div>

In [24]:
# Specify the solver
solver = pyo.SolverFactory('ipopt')

# Solve
results = solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

<div class="admonition note">
<p class="title"><b>Activity</b></p>
 Does your degree of freedom analysis match Ipopt?
</div>


```{dropdown} Solution
Add Solution HERE
```


### Analyze Results

Let's extract the solution.

<div class="admonition note">
<p class="title"><b>Activity</b></p>
 Write Python code to extract and print the solution.
</div>

In [26]:
# Add your solution here
for f in m.FOOD:
    print("Units of",f,"eaten =",round(m.food_eaten[f](),2))

Units of takeaway eaten = 0.0
Units of vegetables eaten = 2.5
Units of bread eaten = 8.0


```{dropdown} Click here to see the activity solution.

```python
for f in m.FOOD:
    print("Units of",f,"eaten =",round(m.food_eaten[f](),2))

```

Next, let us extract the KKT multipliers.

In [27]:
### Declare all suffixes
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html#exporting-suffix-data

# Ipopt bound multipliers
m.ipopt_zL_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
m.ipopt_zU_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Ipopt constraint multipliers
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

# Resolve the model
results = solver.solve(m, tee=True)


Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [ ]:
#Inspect dual variables for lower bound
m.ipopt_zL_out.display()

ipopt_zL_out : Direction=IMPORT, Datatype=FLOAT
    Key                    : Value
         food_eaten[bread] :   8.35310647704211e-10
      food_eaten[takeaway] : 2.5068018506518723e-09
    food_eaten[vegetables] : 1.6730627698360893e-09


In [28]:
#Inspect dual variables for upper bound
m.ipopt_zU_out.display()

ipopt_zU_out : Direction=IMPORT, Datatype=FLOAT
    Key : Value


In [29]:
# Inspect duals
m.dual.display()

dual : Direction=IMPORT_EXPORT, Datatype=FLOAT
    Key         : Value
    diet_min[C] : 1.1385845178627563e-10
    diet_min[F] :     1.9999999967000746
    diet_min[P] :  5.062772006707883e-09
    diet_min[V] :    0.24999999842670498


## Take Away Messages
* Linear programs are convex. We will learn this means all local optima are global optima.
* There are specialize solves for linear programs, quadratic programs, and convex programs. In this class, we will focus on more general algorithms for (non)convex nonlinear programs including the algorithms used by the `ipopt` solver.